In [2]:
import pandas as pd
import numpy as np
import datetime

In [2]:
import os
os.getcwd()

'/home/jian/Projects/Smoothie_King/Dallas_Map'

In [3]:
data_by_store=pd.read_excel("/home/jian/Projects/Smoothie_King/Dallas_Map/SmoothieKing_TA_revised_3_miles_JC_11052018.xlsx",
                           sheetname="output_TA_by_store",dtype=str)


In [4]:
data_zip_long=pd.read_excel("/home/jian/Projects/Smoothie_King/Dallas_Map/SmoothieKing_TA_revised_3_miles_JC_11052018.xlsx",
                           sheetname="zip_TA",dtype=str)
data_zip_long.columns=['TA','Store','zip_cd_3_miles']

In [3]:
DMA_Zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
DMA_Zip=DMA_Zip.iloc[:,[0,2]]
DMA_Zip.columns=['zip_cd','DMA']
DMA_Zip_Dallas=DMA_Zip[DMA_Zip['DMA']=="DALLAS-FT. WORTH"].drop_duplicates()



In [6]:
TAs_associated_with_Dallas=data_zip_long[data_zip_long['zip_cd_3_miles'].isin(DMA_Zip_Dallas['zip_cd'])]
TAs_associated_with_Dallas_list=TAs_associated_with_Dallas['TA'].unique().tolist()

In [7]:
data_zip_long_Dallas=data_zip_long[data_zip_long['TA'].isin(TAs_associated_with_Dallas_list)]
data_zip_long_Dallas=data_zip_long_Dallas[['TA','zip_cd_3_miles']].drop_duplicates()
data_zip_long_Dallas['DMA_of_zip']=np.where(data_zip_long_Dallas['zip_cd_3_miles'].isin(DMA_Zip_Dallas['zip_cd']),"DALLAS-FT. WORTH","Others")

In [8]:
data_by_store_Dallas=data_by_store[data_by_store['TA'].isin(TAs_associated_with_Dallas_list)]
data_by_store_Dallas=data_by_store_Dallas[['storenumber','TA','Latitude','Longitude','city','revised_store_zip','state']]
data_by_store_Dallas.columns=['storenumber','TA']+["store_"+x for x in data_by_store_Dallas.columns.tolist()[2:]]
data_by_store_Dallas['DMA_of_zip']=np.where(data_by_store_Dallas['store_revised_store_zip'].isin(DMA_Zip_Dallas['zip_cd']),"DALLAS-FT. WORTH","Others")



In [9]:
data_for_tableau=data_by_store_Dallas.append(data_zip_long_Dallas)

data_for_tableau['store_Latitude']=data_for_tableau['store_Latitude'].fillna(0)
data_for_tableau['store_Longitude']=data_for_tableau['store_Longitude'].fillna(0)

data_for_tableau['store_Latitude']=data_for_tableau['store_Latitude'].astype(float)
data_for_tableau['store_Longitude']=data_for_tableau['store_Longitude'].astype(float)

In [10]:
data_for_tableau.head(2)

,DMA_of_zip,TA,store_Latitude,store_Longitude,store_city,store_revised_store_zip,store_state,storenumber,zip_cd_3_miles
49,DALLAS-FT. WORTH,024_2,32.910011,-96.802320,Dallas,75230,TX,101,NaN
66,DALLAS-FT. WORTH,032,32.706763,-97.360794,Fort Worth,76109,TX,153,NaN


In [11]:
data_for_tableau['DMA_of_zip'].unique()

array(['DALLAS-FT. WORTH'], dtype=object)

In [12]:
DMA_Zip_NonDallas=DMA_Zip[DMA_Zip['DMA']!="DALLAS-FT. WORTH"].drop_duplicates()
print("stores_not_in_dallas (also in others): ",data_by_store_Dallas[data_by_store_Dallas['store_revised_store_zip'].isin(DMA_Zip_NonDallas['zip_cd'])].shape[0])
print("zips_not_in_dallas (also in others): ",data_zip_long_Dallas[data_zip_long_Dallas['zip_cd_3_miles'].isin(DMA_Zip_NonDallas['zip_cd'])].shape[0])


stores_not_in_dallas (also in others):  0
zips_not_in_dallas (also in others):  0


In [13]:
data_for_tableau.to_csv('/home/jian/Projects/Smoothie_King/Dallas_Map/SK_Dallas_Map_Data_Tableau_JL_'+str(datetime.datetime.now().date())+".csv",index=False)